In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from joblib import dump
import os
import time

In [2]:
CSV_FILE = 'IoMT_train.csv'         # <-- Change to your CSV file
TARGET_COLUMN = 'label'

In [3]:
df = pd.read_csv(CSV_FILE)

In [4]:
label_encoders = {}
for col in df.columns:
    if df[col].dtype == 'object' or df[col].dtype.name == 'category':
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le

# 3. Split features and target
X = df.drop(TARGET_COLUMN, axis=1)
y = df[TARGET_COLUMN]

In [5]:
# 4. Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

from imblearn.over_sampling import SMOTE

# Find the minimum class count in your y_train
min_class_count = y_train.value_counts().min()

# Set k_neighbors to min_class_count - 1 (but at least 1)
k_neighbors = max(1, min_class_count - 1)

smote = SMOTE(random_state=32, k_neighbors=k_neighbors)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [6]:
clf1 = RandomForestClassifier(n_estimators=100, random_state=42)
clf2 = GradientBoostingClassifier(n_estimators=100, random_state=42)
clf3 = LogisticRegression(max_iter=1000, random_state=42)
ensemble = VotingClassifier(estimators=[
    ('rf', clf1),
    ('gb', clf2),
    ('lr', clf3)
], voting='soft')

# 7. Train
ensemble.fit(X_train_res, y_train_res)

C:\Users\akhan\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('gb',
                              GradientBoostingClassifier(random_state=42)),
                             ('lr',
                              LogisticRegression(max_iter=1000,
                                                 random_state=42))],
                 voting='soft')

In [7]:
y_pred = ensemble.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f'Ensemble Accuracy: {acc:.4f}')

Ensemble Accuracy: 0.9435


In [9]:
import joblib
joblib.dump(ensemble, 'iomt2_model.joblib', compress=('xz', 3))

['iomt2_model.joblib']